# Financial Health Dashboard

In [1]:
import pandas as pd
import zipfile

In [2]:
# Step 1: Load the CSV file from the zip archive
zip_file_path = 'Financial Performance of Companies.zip'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()  # Extract all files into the current directory

# Load the CSV file into a pandas DataFrame
csv_file_path = 'snp500_companies_description.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(df.head())
print(df.describe())
print(df.info())

  Company Major index membership Market capitalization Income (ttm)  \
0       A                S&P 500                41.05B        1.32B   
1     AAL                S&P 500                10.65B      127.00M   
2     AAP                S&P 500                 7.88B      501.90M   
3    AAPL     DJIA, NDX, S&P 500              2336.71B       95.17B   
4    ABBV                S&P 500               261.85B       11.78B   

  Revenue (ttm) Book value per share (mrq) Cash per share (mrq)  \
0         6.93B                      18.95                    -   
1        48.97B                      -8.91                13.97   
2        11.15B                      45.06                 4.52   
3       387.54B                       3.57                 3.36   
4        58.05B                       9.74                 5.27   

  Dividend (annual) Dividend yield (annual) Full time employees  ...  \
0              0.90                   0.64%               18300  ...   
1                 -       

In [3]:
# Data Preparation and Cleaning

In [4]:
# Step 1: Convert financial columns (e.g., Market capitalization, Income, Revenue) from strings to numeric
def convert_financial_values(value):
    """Converts string financial values like '41.05B' or '10.65M' into float."""
    if pd.isna(value) or value == '-' or value == '':
        return 0  # Replace missing or invalid values with 0
    # If the value is already a float or int, return it as is
    if isinstance(value, (int, float)):
        return value
    # Handle string values with 'B' or 'M'
    if 'B' in value:
        return float(value.replace('B', '').replace(',', '')) * 1e9
    if 'M' in value:
        return float(value.replace('M', '').replace(',', '')) * 1e6
    # Default: remove commas and convert to float
    return float(value.replace(',', ''))

# Apply conversion to relevant financial columns
financial_columns = ['Market capitalization', 'Income (ttm)', 'Revenue (ttm)', 'Book value per share (mrq)', 'Cash per share (mrq)']
for col in financial_columns:
    df[col] = df[col].apply(convert_financial_values)

In [5]:
# Step 2: Convert percentage columns by handling multiple values and removing '%' symbol
def convert_percentage(value):
    """Converts string percentage values like '9.32%' into float. Handles multiple values like '2.37% 2.09%' by averaging."""
    if pd.isna(value) or value == '-' or value == '':
        return 0  # Replace missing or invalid values with 0
    # If the value is already a float or int, return it as is
    if isinstance(value, (int, float)):
        return value
    # Remove the '%' symbol before processing
    value = value.replace('%', '')
    # Handle cases where there are two percentage values (e.g., '2.37 2.09')
    if ' ' in value:
        values = [float(v) for v in value.split()]
        return sum(values) / len(values)  # Return the average of the two values
    # Handle single percentage values
    return float(value)

# Apply conversion to relevant percentage columns
percentage_columns = ['Dividend yield (annual)', 'Performance (Quarter)', 'Performance (Half Year)', 'Performance (Year)', 'Performance (Year To Date)', 'Volatility (Week, Month)', 'Performance (today)']
for col in percentage_columns:
    df[col] = df[col].apply(convert_percentage)

In [6]:
# Step 3: Convert ratio columns from strings to numeric (e.g., Current Ratio, Quick Ratio)
ratio_columns = ['Current Ratio (mrq)', 'Quick Ratio (mrq)', 'Total Debt to Equity (mrq)', 'Long Term Debt to Equity (mrq)']
for col in ratio_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, set errors to NaN

# Step 4: Handle missing or invalid values (e.g., '-' or missing values)
df.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity

# Step 5: Convert any other relevant columns (e.g., 'Price-to-Earnings (ttm)') from strings to float where necessary
df['Price-to-Earnings (ttm)'] = pd.to_numeric(df['Price-to-Earnings (ttm)'], errors='coerce')

# Display cleaned data info and a sample to verify the changes
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 73 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   Company                                                                    496 non-null    object 
 1   Major index membership                                                     496 non-null    object 
 2   Market capitalization                                                      496 non-null    float64
 3   Income (ttm)                                                               496 non-null    float64
 4   Revenue (ttm)                                                              496 non-null    float64
 5   Book value per share (mrq)                                                 496 non-null    float64
 6   Cash per share (mrq)                                      

In [7]:
# Preparing Data for Tableau Visualizations

In [8]:
# Step 1: Assume or create mock 'Expenses', 'Current Assets', and 'Current Liabilities' columns
# For demonstration, we'll assume Expenses are 80% of Revenue
df['Expenses'] = df['Revenue (ttm)'] * 0.8  # Example: Assuming 80% of revenue is expenses

# For Current Assets and Liabilities, create mock values (replace with real values if available)
df['Current Assets'] = df['Revenue (ttm)'] * 0.6  # Assuming 60% of revenue is current assets
df['Current Liabilities'] = df['Revenue (ttm)'] * 0.4  # Assuming 40% of revenue is current liabilities
df['Inventory'] = df['Revenue (ttm)'] * 0.2  # Assuming 20% of revenue is inventory

# Step 2: Calculate Profit (Profit = Revenue - Expenses)
df['Profit'] = df['Revenue (ttm)'] - df['Expenses']

# Step 3: Calculate Financial Ratios
df['Current Ratio'] = df['Current Assets'] / df['Current Liabilities']
df['Quick Ratio'] = (df['Current Assets'] - df['Inventory']) / df['Current Liabilities']

# Step 4: Prepare Data for Tableau

# 1. Profit and Loss Statement (For Line Chart in Tableau)
# Grouping by 'Company' to see overall financial performance (you can add Date if time-based data exists)
profit_loss_over_time = df[['Company', 'Revenue (ttm)', 'Expenses', 'Profit']]

# 2. Revenue vs. Expenses (For Bar Chart in Tableau)
# Grouping by 'Company' to compare revenue and expenses
revenue_vs_expenses = df[['Company', 'Revenue (ttm)', 'Expenses']]

# 3. Financial Ratios (For Gauge Chart in Tableau)
# Keep necessary financial ratios for visualization
financial_ratios = df[['Company', 'Current Ratio', 'Quick Ratio']]

In [9]:
# Step 5: Save Prepared Data for Tableau

# Save profit and loss statement
profit_loss_csv = 'profit_loss_over_time.csv'
profit_loss_over_time.to_csv(profit_loss_csv, index=False)

# Save revenue vs expenses data
revenue_vs_expenses_csv = 'revenue_vs_expenses.csv'
revenue_vs_expenses.to_csv(revenue_vs_expenses_csv, index=False)

# Save financial ratios data
financial_ratios_csv = 'financial_ratios.csv'
financial_ratios.to_csv(financial_ratios_csv, index=False)

print(f"Data prepared for Tableau visualization:")
print(f"Profit and Loss: {profit_loss_csv}")
print(f"Revenue vs Expenses: {revenue_vs_expenses_csv}")
print(f"Financial Ratios: {financial_ratios_csv}")

Data prepared for Tableau visualization:
Profit and Loss: profit_loss_over_time.csv
Revenue vs Expenses: revenue_vs_expenses.csv
Financial Ratios: financial_ratios.csv


In [10]:
# You can then use Tableau to build the visualizations:
# Profit and Loss Statement (Line Chart),
# Revenue vs. Expenses (Bar Chart),
# Financial Ratios (Gauge Chart).